In [16]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'DB_and_Azure'))
import sql_db_functions as SQLf
import Chroma_db_functions as Cf

from apikey import apikey 

from langchain.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

from chromadb.config import Settings
import uuid

In [17]:
persist_directory = 'chroma-db'
embedding = Cf.get_embeddings()

In [18]:
db = Chroma(persist_directory= persist_directory, embedding_function=embedding,collection_name='encodings', )


In [54]:
#Ensure we are pulling data correctly
db.similarity_search(query='hello')

[Document(page_content='Brand: HM / Type: Dress   / Fit: Regular fit   / Sleeve style: Long sleeve   / Neckline: V-neck   / Material: Cotton   / Formality: Casual   / Seasson: Autumn   / Colors: 100% Black   / Texture: Smooth   / Transparency: Opaque   / Details and Embellishments: Buttons   / Shape: Fitted   / Length: Midi   / Collar Style: Not applicable   / Sleeve Style: Long   / Patterns: Not applicable   / Patterns placement: Not applicable   / Fluidity of fabric: Moderate   / Fabric weight: Medium   / Pocket Presence: No   / Pocket placement: Not applicable   / Pocket size: Not applicable   / Breathability: Medium   / Occasion Suitability: Casual   / Lapel: Not applicable', metadata={'source': '19'}),
 Document(page_content='Brand: HM / Type: Dress / Fit: Regular fit / Sleeve style: Long sleeve / Neckline: V-neck / Material: Polyester / Formality: Casual / Season: Autumn, Spring / Colors: Black 90%, White 10% / Texture: Smooth / Transparency: Opaque / Details and Embellishments: 

In [20]:
retriever = db.as_retriever(search_kwargs={"k": 5})

In [21]:
os.environ['OPENAI_API_KEY'] = apikey

In [22]:
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

c:\Users\juand\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


### Test chain 
Setting up -> https://python.langchain.com/v0.2/docs/tutorials/rag/

In [23]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [32]:
from langchain import hub

prompt_hub = hub.pull("rlm/rag-prompt")

In [33]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [34]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_hub
    | turbo_llm
    | StrOutputParser()
)

In [29]:
### lets set up the system

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [48]:
system_prompt = (
    "You are a search engine for clothing. "
    "Use the following pieces of retrieved context to suggest the best peace of clothing "
    "the question. If you don't know the answer, say that you "
    "answer concise."
    "Explain why the 5 selected options of the context are the best ones"
    "\n\n"
    "{context}"
)

In [49]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [50]:
question_answer_chain = create_stuff_documents_chain(turbo_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [42]:
query = "Brand: HM / Type: Dress   / Fit: Regular fit   / Sleeve style: Long sleeve   / Neckline: V-neck   / Material: Cotton   / Formality: Casual   / Seasson: Autumn   / Colors: 100% Black   / Texture: Smooth   / Transparency: Opaque   / Details and Embellishments: Buttons   / Shape: Fitted   / Length: Midi   / Collar Style: Not applicable   / Sleeve Style: Long   / Patterns: Not applicable   / Patterns placement: Not applicable   / Fluidity of fabric: Moderate   / Fabric weight: Medium   / Pocket Presence: No   / Pocket placement: Not applicable   / Pocket size: Not applicable   / Breathability: Medium   / Occasion Suitability: Casual   / Lapel: Not applicable'"

In [51]:
query = "/ Material: Cotton   / Formality: Casual   / Seasson: Autumn   / Colors: 100% Black   / Texture: Smooth   / Transparency: Opaque   / Details and Embellishments: Buttons / Sleeve Style: Long / Patterns placement: Not applicable   / Fluidity of fabric: Moderate   / Fabric weight: Medium   / Pocket Presence: No   / Pocket placement: Not applicable   / Pocket size: Not applicable   / Breathability: Medium   / Occasion Suitability: Casual   / Lapel: Not applicable'"

In [52]:
response = rag_chain.invoke({"input": query})
print(response["answer"])

Based on the provided context, the best option for you would be the dress described as follows:

- Brand: HM
- Type: Dress
- Fit: Regular fit
- Sleeve style: Long sleeve
- Neckline: V-neck
- Material: Cotton
- Formality: Casual
- Season: Autumn
- Color: Black
- Texture: Smooth
- Transparency: Opaque
- Details and Embellishments: Buttons
- Shape: Fitted
- Length: Midi
- Sleeve Style: Long
- Patterns: Not applicable
- Fluidity of fabric: Moderate
- Fabric weight: Medium
- Pocket Presence: No
- Pocket placement: Not applicable
- Pocket size: Not applicable
- Breathability: Medium
- Occasion Suitability: Casual
- Lapel: Not applicable

This dress is a great choice for autumn due to its cotton material, which provides moderate breathability and medium fabric weight, making it suitable for the season. The black color, smooth texture, and fitted shape add to its casual formality, while the V-neck and long sleeves make it versatile for various occasions. The absence of patterns and pockets kee

In [53]:
for document in response["context"]:
    print(document)
    print()

page_content='Brand: HM / Type: Dress / Fit: Regular fit / Sleeve style: Long sleeve / Neckline: V-neck / Material: Likely polyester or a blend / Formality: Casual, Business Casual / Season: Autumn, Spring / Colors: Black (80%), White (20%) / Texture: Smooth / Transparency: Opaque / Details and Embellishments: Buttons / Shape: Fitted / Length: Midi / Collar Style: Not applicable / Sleeve Style: Long / Patterns: Polka dots / Patterns placement: Throughout / Fluidity of fabric: Medium / Fabric weight: Medium / Pocket Presence: No / Pocket placement: Not applicable / Pocket size: Not applicable / Breathability: Medium / Occasion Suitability: Casual, business casual / Lapel: Not applicable' metadata={'source': '17'}

page_content='Brand: HM / Type: Dress / Fit: Regular fit, Loose / Sleeve style: Long sleeve / Neckline: V-neck / Material: Polyester / Formality: Casual, Business Casual / Season: Autumn, Spring / Colors: Black (90%), White (10%) / Texture: Smooth / Transparency: Opaque / Deta